In [84]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import brickschema
import datetime
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.similarity_score import *
import uuid
import pickle
from rdflib import URIRef, Literal, BNode
from queries.mortar import queries as q1
import time
%matplotlib inline

In [2]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 32}

matplotlib.rc('font', **font)

In [3]:
useful_queries = [
"""SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Zone_Air_Temperature_Sensor .
    ?sensor brick:isPointOf ?equip 
}""",
"""SELECT ?sp WHERE {
    ?sp rdf:type brick:Zone_Air_Temperature_Setpoint .
    ?sp brick:isPointOf ?equip
}""",
"""SELECT ?sensor ?sp ?equip WHERE {
    ?sensor  rdf:type     brick:Air_Flow_Sensor .
    ?sp      rdf:type     brick:Air_Flow_Setpoint .
    ?sensor  brick:isPointOf ?equip .
    ?sp      brick:isPointOf ?equip
}""",
"""SELECT * WHERE {
    ?equip    rdf:type  brick:VAV .
    ?equip    brick:hasPoint  ?air_flow .
    ?air_flow rdf:type   brick:Supply_Air_Flow_Sensor
    }""",
"""SELECT ?vav WHERE {
    ?vav rdf:type brick:VAV
}""",
"""SELECT ?floor ?room WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?room brick:isPartOf+ ?floor .
}""",
    
"""SELECT * WHERE {
    ?oat_damper a brick:Outside_Damper .
    ?pos a brick:Damper_Position_Command .
    ?oat_damper brick:hasPoint ?pos .
    ?oat a brick:Outside_Air_Temperature_Sensor
}""",
    
"""SELECT * WHERE {
    ?equip   rdf:type  brick:VAV .
    ?equip   brick:isFedBy  ?ahu .
    ?ahu     brick:hasPoint  ?upstream_ta .
    ?equip   brick:hasPoint  ?dnstream_ta .
    ?upstream_ta  rdf:type brick:Supply_Air_Temperature_Sensor .
    ?dnstream_ta  rdf:type brick:Supply_Air_Temperature_Sensor .
    ?equip   brick:hasPoint  ?vlv .
    ?vlv     rdf:type   brick:Valve_Command
}"""
]

In [7]:
useful_buildings = [31, 35, 37]

files = os.listdir('Data - Brick models/')
brick_models = []

with open("brick_models.pkl", 'rb') as fp:
    brick_models = pickle.load(fp)
    

## max_level = 3

In [13]:
results = {}
output_queries = {}
for query in useful_queries:
    max_level = 3
    print_counter = 3
    G = get_fast_relaxed_graph(query=query, max_level=max_level)
    already_done_queries = []
    original_uuid = list(G.nodes())[0]
    original_query = G.nodes()[original_uuid]['query']
    results_query = {}
    output_queries_buildings = {}
    select_statement = query.split("{")[0] + "{\n"
    
    for building in useful_buildings:
        g = brick_models[building]
        total_instances = get_total_num_instances(g, 'brick:Equipment') + get_total_num_instances(g, 'brick:Location') + get_total_num_instances(g, 'brick:Point')
        print(total_instances)
        scores = {}
        results_building = {}
        for idx, node in G.nodes().data():    
            relaxed_query = list(node['query'])
            score = sim_score(original_query, relaxed_query, g, total_instances)

            level = node['level']
            if score in scores:
                scores[score].append(relaxed_query)
            else:
                scores[score] = [relaxed_query]
                
        max_score = 0
        count = 0
        queries_to_print1 = []

        for score in sorted(list(scores.keys()), reverse = True):
            if score > max_score:
                max_score = score

            for i in scores[score]:
                if not str(i) in results_building:
                    brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
                    res = run_brick_query(building_model=g, query=brick_query)
                    results_building[str(i)] = res
                else:
                    res = results_building[str(i)]

                if len(res) > 0:
                    if not (i, score) in queries_to_print1:
                        queries_to_print1.append((i, score))

            count = len(queries_to_print1)

            if count >= print_counter:
                break
        output_queries_buildings[building] = queries_to_print1
        results_query[building] = results_building
        
        print("building {} done".format(building))
    print("\n")
        
    output_queries[str(query)] = results_query
    results[str(query)] = results_query

221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done


221
building 31 done
1412
building 35 done
52
building 37 done




In [77]:
def get_query_from_str(relaxed_q):
    rel_qs = []
    for s in relaxed_q.split('['):
        if ']' in s:
            s2 = s.split(']')[0]
            rel_q = []
            for s3 in s2.split(", "):
                rel_q.append(s3.replace("'", ""))
            rel_qs.append(rel_q)
    return rel_qs

In [81]:
for q in output_queries:
    print("original query: {}".format(q))
    G = get_fast_relaxed_graph(query=q, max_level=max_level)
    original_uuid = list(G.nodes())[0]
    original_query = G.nodes()[original_uuid]['query']
    for b in output_queries[q]:
        print("building: {}".format(b))
        g = brick_models[b]
        total_instances = get_total_num_instances(g, 'brick:Equipment') + get_total_num_instances(g, 'brick:Location') + get_total_num_instances(g, 'brick:Point')
        op = []        
        for relaxed_q in output_queries[q][b]:

            res = output_queries[q][b][relaxed_q]

            if len(res) > 0:
                score = sim_score(original_query, get_query_from_str(relaxed_q), g, total_instances)
#                 print(relaxed_q, score)
                if len(op) <3:
                    op.append(score)
            
        print(round(np.mean(op), 2), len(op))
    print("\n")

original query: SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Zone_Air_Temperature_Sensor .
    ?sensor brick:isPointOf ?equip 
}
building: 31
0.98 3
building: 35
0.99 3
building: 37
0.67 3


original query: SELECT ?sp WHERE {
    ?sp rdf:type brick:Zone_Air_Temperature_Setpoint .
    ?sp brick:isPointOf ?equip
}
building: 31
1.0 3
building: 35
1.0 3
building: 37
0.67 3


original query: SELECT ?sensor ?sp ?equip WHERE {
    ?sensor  rdf:type     brick:Air_Flow_Sensor .
    ?sp      rdf:type     brick:Air_Flow_Setpoint .
    ?sensor  brick:isPointOf ?equip .
    ?sp      brick:isPointOf ?equip
}
building: 31
0.67 3
building: 35
0.67 3
building: 37
nan 0


original query: SELECT * WHERE {
    ?equip    rdf:type  brick:VAV .
    ?equip    brick:hasPoint  ?air_flow .
    ?air_flow rdf:type   brick:Supply_Air_Flow_Sensor
    }
building: 31
1.0 3
building: 35
1.0 3
building: 37
nan 0


original query: SELECT ?vav WHERE {
    ?vav rdf:type brick:VAV
}
building: 31
0.98 3
building: 35
0.9